[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/mohsennasab/python-fundamentals-hh/blob/main/notebooks/02_timeseries/02_01_TimeSeries_Basics.ipynb
)

# Module 2, Lesson 1: Time Series Analysis for H&H with Python

## Working with USGS daily discharge data (Mississippi River at St. Paul, MN)

---

### Welcome!

In this lesson, you'll learn the time series skills you'll use constantly in hydrology and hydraulics: reading data, parsing dates, resampling, unit conversions, plotting hydrographs, and calculating common summary statistics.

We'll use a real dataset: **USGS daily discharge (cfs)** for **USGS 05331000** (Mississippi River at St. Paul, Minnesota).

---

### 🤖 About AI Assistance in This Course

Throughout this lesson, you'll see **"🤖 AI Assistant Prompt"** boxes. These are ready-to-use prompts you can copy and paste into your favorite AI assistant (ChatGPT, Claude, Gemini, Copilot, etc.) to:

- Get explanations of code you don't understand
- Debug errors you encounter
- Modify code for your own projects
- Learn the "why" behind each step

**Using AI is not cheating—it's a professional skill.** Engineers use AI assistants daily. The goal is to understand what the code does, not memorize every syntax detail.

> **Note:** In this module we use **streamflow (discharge)** as our example time series. The same workflows apply to most water-resources time series: **precipitation**, **snow water equivalent**, **water level/stage**, **groundwater levels**, **water temperature**, **turbidity**, and **water quality** measurements. Once you learn these skills, you can reuse them across datasets.

### What you'll be able to do by the end

After completing this lesson, you will be able to:

1. **Read a CSV time series** and parse datetime correctly
2. **Set a datetime index** and do time-based slicing (e.g., "give me just 2020 data")
3. **Handle missing values** and common data issues
4. **Convert units** (cfs ↔ cms) and aggregate to different time steps (daily → monthly)
5. **Make common plots** for time series: Hydrograph, Flow Duration Curve, Box Plots
6. **Compute basic statistics** (min/mean/max, percentiles, annual metrics)
7. **Use AI as a pair programmer** to explain, write, debug, and improve your code

---

## 🤖 How to Use AI Assistants Effectively

### The Three Types of AI Prompts You'll Use

| Type | When to Use | Example |
|------|-------------|--------|
| **Explain** | You see code and want to understand it | "Explain this code line by line..." |
| **Debug** | Something isn't working | "I got this error... what's wrong?" |
| **Modify** | You want to adapt code for your needs | "How do I change this to use my data?" |

### Best Practices

1. **Always include the code** when asking questions
2. **Include the full error message** when debugging
3. **Describe your goal**, not just what you tried
4. **Ask follow-up questions** if the explanation isn't clear

---

### 🤖 AI Assistant Prompts for This Module

**For understanding code:**
```
I'm learning Python for hydrology. Explain what this code is doing line by line,
and point out any assumptions. Use simple language:

[paste code here]
```

**For debugging:**
```
I'm working in Google Colab on a hydrology project. I got this error:

[paste error message]

Here's my code:

[paste code]

What does this error mean and how do I fix it?
```

**For modifying code:**
```
I have this Python code that works with discharge data:

[paste code]

How would I modify it to [describe your goal]?
```

### Lesson Structure

1. **Mental models** for time series (connecting to concepts you already know)
2. **Load and clean** USGS discharge data
3. **Datetimes**: parsing, time zones, indexing, slicing
4. **Resampling and aggregation** (daily → weekly/monthly)
5. **Visualization patterns** you'll reuse in reports
6. **Practical statistics** for H&H
7. **Mini-project**: a clean "hydrograph summary" workflow
8. **Practice exercises** (with AI prompts for help)

---

## Part 1: Mental Models for Time Series

### What is a Time Series? (The Excel Analogy)

If you've worked with Excel, you already understand time series data. It's simply a table where:
- **One column contains dates/times** (when the measurement was taken)
- **Another column contains values** (what was measured)

Think of it like a gage record from USGS:

| Date | Discharge (cfs) |
|------|----------------|
| 2024-01-01 | 5,000 |
| 2024-01-02 | 5,200 |
| 2024-01-03 | 4,800 |

### Why Time Series Needs Special Handling

In hydrology, almost everything is "value vs time":
- **Rainfall hyetographs** (precipitation rate over time)
- **Streamflow hydrographs** (discharge over time)
- **Stage records** (water level over time)
- **Snow water equivalent (SWE)** over a season

In Python (using pandas), your job is to:

1. **Make sure time is truly a datetime type** (not just text)
2. **Make time the index** (so slicing and resampling work cleanly)
3. **Keep units and time step clear** (daily vs hourly vs 15-min)

### ⚠️ A Helpful Rule

> If your time column is still "object" or "string" type, treat it as **not ready** for analysis!

---

### 🤖 AI Assistant Prompt: Understanding Time Series

```
I'm a water resources engineer learning Python. Explain what a "datetime index" is
in pandas and why it's useful for hydrologic time series data.
Compare it to how I might work with dates in Excel.
```

---

## Part 2: Setup (Run Once at the Start)

### What are "Imports"?

Think of imports like loading toolboxes in a workshop. Before you can use a power drill, you need to get it out of storage.

We'll use three main libraries:

| Library | What It Does | Analogy |
|---------|--------------|--------|
| `pandas` | Data manipulation (tables, filtering, aggregating) | Excel on steroids |
| `matplotlib` | Creating plots and charts | Like Excel charts, but more flexible |
| `numpy` | Numerical calculations | A powerful calculator |

> **If you see an import error:** Copy the error message and ask your AI assistant: "What do I need to install in Colab to fix this?"

In [ ]:
# =============================================================================
# IMPORTS - Run this cell first!
# =============================================================================
# These are the "toolboxes" we'll use throughout this lesson.

import pandas as pd      # pd is a nickname (alias) so we type less
import numpy as np       # np is the standard nickname for numpy
import matplotlib.pyplot as plt  # plt is standard for plotting

# These settings make tables display nicely in Colab
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)

print("✓ All libraries loaded successfully!")
print(f"  - pandas version: {pd.__version__}")
print(f"  - numpy version: {np.__version__}")

### 🔍 Understanding the Code Above

```python
import pandas as pd
```

This line:
1. **`import pandas`** - Loads the pandas library
2. **`as pd`** - Creates a shorter nickname

Instead of typing `pandas.read_csv()`, we can now type `pd.read_csv()`

---

### 🤖 AI Assistant Prompt: Understanding Imports

```
I'm new to Python. Explain what these import statements do and why we use
nicknames like 'pd' and 'np':

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
```

---

## Part 3: Loading the USGS Dataset 📥

### About the Data

We're using real USGS daily discharge data for the Mississippi River at St. Paul, MN (Station 05331000).

**What's in the file:**
- ~25 years of daily discharge measurements
- Values in cubic feet per second (cfs)
- Quality flags and metadata

### Getting Data into Colab

We'll show you **two beginner-friendly methods** to load a CSV:

1. **Upload from your computer** (easiest for beginners)
2. **Load from Google Drive** (better for repeated use)

### Method 1: Upload from Your Computer

Run the cell below. A "Choose Files" button will appear. Click it and select your CSV file.

> **First time?** Download the file from the course GitHub repository first.

In [ ]:
# =============================================================================
# METHOD 1: Upload a file from your computer
# =============================================================================

from google.colab import files

print("Click 'Choose Files' below and select your CSV file...")
uploaded = files.upload()

# Get the filename of what was uploaded
uploaded_filename = next(iter(uploaded))
data_path = uploaded_filename

print(f"\n✓ File uploaded successfully!")
print(f"  Filename: {data_path}")

### 🔍 What Just Happened?

1. **`from google.colab import files`** - Loads Colab's file upload tool
2. **`files.upload()`** - Opens a file picker in your browser
3. **`uploaded`** - A dictionary containing your file(s)
4. **`next(iter(uploaded))`** - Gets the name of the first uploaded file

---

### 🤖 AI Assistant Prompt: File Upload Issues

```
I'm using Google Colab and trying to upload a CSV file. The files.upload()
function ran but I got this error:

[paste your error here]

What's wrong and how do I fix it?
```

---

### Reading the CSV File

Now let's load the data into a pandas **DataFrame**. A DataFrame is pandas' version of a spreadsheet—a table with rows and columns.

In [ ]:
# =============================================================================
# READ THE CSV FILE
# =============================================================================
# pd.read_csv() is the main function for loading CSV files.

raw = pd.read_csv(data_path)

# Let's see what we got:
print(f"Dataset shape: {raw.shape[0]} rows × {raw.shape[1]} columns")
print(f"\nColumn names:")
for col in raw.columns:
    print(f"  - {col}")

print(f"\nFirst 5 rows:")
raw.head()

### 🔍 Understanding `pd.read_csv()`

```python
raw = pd.read_csv(data_path)
```

- **`pd.read_csv()`** - Reads a CSV file and creates a DataFrame
- **`data_path`** - The filename (or full path) to your CSV
- **`raw`** - We store the result in a variable called `raw` (for "raw data")

The **`.shape`** attribute tells you dimensions: `(rows, columns)`

The **`.head()`** method shows the first 5 rows

### Quick Data Audit

Before doing any analysis, always check:
- What columns exist?
- What **type** is each column? (especially: is the date a proper datetime?)
- Are there missing values?

The **`.info()`** method gives us all of this at once:

In [ ]:
# =============================================================================
# DATA AUDIT: Check column types and missing values
# =============================================================================
# .info() is your best friend for understanding a new dataset!

raw.info()

### 🔍 Reading the `.info()` Output

Look at the `Dtype` column:

| Dtype | What It Means | Example |
|-------|---------------|--------|
| `int64` | Integer numbers | 5000, -10, 0 |
| `float64` | Decimal numbers | 3.14, -2.5 |
| `object` | Text (strings) | "2024-01-01", "Approved" |
| `datetime64` | Proper dates | (what we want for time!) |

**Notice:** The `time` column is probably showing as `object`, not `datetime64`. This means Python sees it as text, not as a real date. We'll fix this soon!

---

### 🤖 AI Assistant Prompt: Understanding Data Types

```
I ran .info() on my pandas DataFrame and got this output:

[paste your info output here]

Explain what each Dtype means and whether any columns need to be converted
to different types for time series analysis.
```

---

### Creating a Clean Working Table

USGS data files often contain many columns we don't need. Let's:

1. Keep only the columns we need
2. Rename them to something friendlier

**Columns we'll keep:**
- `time` → rename to `date`
- `value` → rename to `Q_cfs` (Q is the standard symbol for discharge)

In [ ]:
# =============================================================================
# CREATE A CLEAN WORKING TABLE
# =============================================================================

# Step 1: Make a copy so we don't modify the original
df = raw.copy()

# Step 2: Rename columns to be more intuitive
df = df.rename(columns={
    "time": "date",
    "value": "Q_cfs"
})

# Step 3: Keep only the columns we need
keep_cols = ["date", "Q_cfs", "unit_of_measure", "approval_status"]
df = df[[c for c in keep_cols if c in df.columns]]

print(f"Clean dataset: {df.shape[0]} rows × {df.shape[1]} columns\n")
df.head()

### 🔍 Understanding the Cleaning Code

```python
df = raw.copy()
```
Creates a copy of `raw`. If we mess up, the original data is still safe.

```python
df = df.rename(columns={"time": "date", "value": "Q_cfs"})
```
The `rename()` function uses a **dictionary** `{}` to map old names to new names.

```python
df = df[["date", "Q_cfs", ...]]
```
This **selects specific columns** using double brackets `[[]]`.

---

## Part 4: Datetime Parsing and Indexing

This is the **most important step** for time series analysis.

### Why Datetimes Matter

Right now, our `date` column is just text (type "object"). Python doesn't know that "2024-01-01" is a date.

We need to **convert** it to a proper datetime so we can:
- Sort by date
- Filter by year, month, or date range
- Calculate time differences
- Resample to different time periods

### Step 1: Convert to Datetime

In [ ]:
# =============================================================================
# CONVERT DATE COLUMN TO DATETIME
# =============================================================================

# BEFORE: Check the type
print(f"BEFORE conversion:")
print(f"  date column type: {df['date'].dtype}")
print(f"  Example value: {df['date'].iloc[0]} (just text!)\n")

# CONVERT: Use pd.to_datetime()
df['date'] = pd.to_datetime(df['date'])

# AFTER: Check the type
print(f"AFTER conversion:")
print(f"  date column type: {df['date'].dtype}")
print(f"  Example value: {df['date'].iloc[0]} (now a real datetime!)\n")

# Check for any failed conversions
missing_dates = df['date'].isna().sum()
print(f"  Missing/invalid dates: {missing_dates}")

### 🔍 Understanding `pd.to_datetime()`

```python
df['date'] = pd.to_datetime(df['date'])
```

This line:
1. **`df['date']`** - Selects the 'date' column
2. **`pd.to_datetime()`** - Converts text to datetime
3. **`df['date'] = ...`** - Saves the result back

Pandas recognizes common date formats automatically!

---

### 🤖 AI Assistant Prompt: Date Format Issues

```
I'm trying to convert a date column to datetime in pandas. My dates look like this:

[paste a few example dates]

But pd.to_datetime() is giving me errors. What format string do I need to use?
```

---

### Step 2: Set Date as the Index

The **index** is like the row labels. For time series, we want the date to be the index because:

1. It makes **time-based slicing** easy: `df['2020']` gets all 2020 data
2. It enables **resampling**: converting daily to monthly data
3. Plots automatically use dates on the x-axis

In [ ]:
# =============================================================================
# SET DATE AS INDEX AND SORT
# =============================================================================

df = df.sort_values('date')  # Sort chronologically
df = df.set_index('date')    # Make date the index

print(f"Index type: {type(df.index).__name__}")
print(f"Date range: {df.index.min().strftime('%Y-%m-%d')} to {df.index.max().strftime('%Y-%m-%d')}")
print(f"\nFirst 5 rows (notice date is now the index, not a column):")
df.head()

### Step 3: Basic Quality Checks

Before analysis, always verify:
- Is the index unique? (no duplicate dates)
- Is the time step consistent?
- Are there gaps in the record?

In [ ]:
# =============================================================================
# QUALITY CHECKS
# =============================================================================

print("=== Data Quality Checks ===")
print(f"\n1. Index unique?: {df.index.is_unique}")
print(f"\n2. Date range: {df.index.min()} to {df.index.max()}")

# Check time step
time_deltas = df.index.to_series().diff().dropna()
print(f"\n3. Most common time step (days):")
print(time_deltas.dt.days.value_counts().head(3))

print(f"\n4. Missing discharge values: {df['Q_cfs'].isna().sum()}")

---

## Part 5: Time-Based Slicing (Filtering by Date)

One of the biggest benefits of a datetime index is easy **slicing**—selecting data for specific time periods.

In [ ]:
# =============================================================================
# TIME-BASED SLICING EXAMPLES
# =============================================================================

# Get all data for a specific year
print("=== All 2020 data (first 5 rows) ===")
print(df.loc['2020'].head())

# Get data for a specific month
print("\n=== June 2020 data (first 5 rows) ===")
print(df.loc['2020-06'].head())

# Get data for a date range
print("\n=== May 1-15, 2020 ===")
print(df.loc['2020-05-01':'2020-05-15'])

### 🔍 Understanding Time-Based Slicing

| Code | Result |
|------|--------|
| `df.loc['2020']` | All rows from year 2020 |
| `df.loc['2020-06']` | All rows from June 2020 |
| `df.loc['2020-06-15']` | Just June 15, 2020 |
| `df.loc['2020':'2022']` | All data from 2020 through 2022 |

This only works **because we set the date as the index**!

---

## Part 6: Unit Conversion (cfs ↔ cms)

**Conversion factor:** 1 cfs = 0.028317 cms

In [ ]:
# =============================================================================
# UNIT CONVERSION: cfs to cms
# =============================================================================

CFS_TO_CMS = 0.028317

# Create a new column with converted values
df['Q_cms'] = df['Q_cfs'] * CFS_TO_CMS

print("Discharge in both units (first 5 rows):")
print(df[['Q_cfs', 'Q_cms']].head())

print(f"\nMean discharge: {df['Q_cfs'].mean():,.0f} cfs = {df['Q_cms'].mean():,.1f} cms")

---

## Part 7: Resampling (Changing Time Steps)

Often you need to **aggregate** data to different time periods:

| Code | Meaning |
|------|--------|
| `'D'` | Daily |
| `'W'` | Weekly |
| `'MS'` | Month Start |
| `'YS'` | Year Start |

In [ ]:
# =============================================================================
# RESAMPLE TO MONTHLY DATA
# =============================================================================

monthly_mean = df['Q_cfs'].resample('MS').mean()

print("Monthly Mean Discharge (first 12 months):")
print(monthly_mean.head(12))

print(f"\nOriginal: {len(df)} daily values")
print(f"After resampling: {len(monthly_mean)} monthly values")

### 🔍 Understanding `.resample()`

```python
monthly_mean = df['Q_cfs'].resample('MS').mean()
```

Think of it as a two-step process:
1. **`resample('MS')`** - Groups data by month
2. **`.mean()`** - Calculates the average for each group

You can replace `.mean()` with `.max()`, `.min()`, `.sum()`, `.median()`

In [ ]:
# =============================================================================
# ROLLING AVERAGES
# =============================================================================

# Calculate 7-day and 30-day rolling means
df['Q_7day'] = df['Q_cfs'].rolling(window=7).mean()
df['Q_30day'] = df['Q_cfs'].rolling(window=30).mean()

print("Daily vs Rolling Averages (sample period):")
print(df.loc['2020-05-01':'2020-05-15', ['Q_cfs', 'Q_7day', 'Q_30day']])

---

## Part 8: Visualization Patterns for H&H

### Plot 1: Basic Hydrograph

In [ ]:
# =============================================================================
# PLOT 1: BASIC HYDROGRAPH
# =============================================================================

fig, ax = plt.subplots(figsize=(12, 5))

ax.plot(df.index, df['Q_cfs'], linewidth=0.5, color='steelblue')

ax.set_xlabel('Date')
ax.set_ylabel('Discharge (cfs)')
ax.set_title('Mississippi River at St. Paul, MN - Daily Discharge')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 🔍 Understanding Matplotlib Plotting

```python
fig, ax = plt.subplots(figsize=(12, 5))
```
- **`fig`** = the overall figure (like a canvas)
- **`ax`** = the axes where we draw
- **`figsize=(12, 5)`** = width and height in inches

```python
ax.plot(df.index, df['Q_cfs'], linewidth=0.5, color='steelblue')
```
- First argument = x-values (dates)
- Second argument = y-values (discharge)

---

### 🤖 AI Assistant Prompt: Customizing Plots

```
I made this matplotlib plot:

[paste your plotting code]

I want to modify it to [describe changes, e.g., "add a horizontal line at 10,000 cfs",
"change the x-axis to only show 2020-2022"].

Show me how.
```

---

### Plot 2: Monthly Box Plot (Seasonal Patterns)

In [ ]:
# =============================================================================
# PLOT 2: MONTHLY BOX PLOT
# =============================================================================

df['month'] = df.index.month

fig, ax = plt.subplots(figsize=(10, 5))
df.boxplot(column='Q_cfs', by='month', ax=ax)

ax.set_xlabel('Month')
ax.set_ylabel('Discharge (cfs)')
ax.set_title('Seasonal Distribution of Discharge')
fig.suptitle('')

month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
ax.set_xticklabels(month_names)

plt.tight_layout()
plt.show()

### Plot 3: Flow Duration Curve (FDC)

The **Flow Duration Curve** shows what discharge is exceeded X% of the time.

In [ ]:
# =============================================================================
# PLOT 3: FLOW DURATION CURVE
# =============================================================================

q = df['Q_cfs'].dropna().values
q_sorted = np.sort(q)[::-1]
n = len(q_sorted)
exceedance = np.arange(1, n+1) / (n+1) * 100

fig, ax = plt.subplots(figsize=(8, 5))

ax.plot(exceedance, q_sorted, color='steelblue', linewidth=1)

ax.set_xlabel('Exceedance Probability (%)')
ax.set_ylabel('Discharge (cfs)')
ax.set_title('Flow Duration Curve')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 🔍 Understanding the Flow Duration Curve

**How to read it:**
- **X-axis** = Exceedance probability (% of time flow is exceeded)
- **Y-axis** = Discharge
- **Q10** = Flow exceeded 10% of time (high flow)
- **Q50** = Flow exceeded 50% of time (median)
- **Q90** = Flow exceeded 90% of time (low flow)

---

## Part 9: Practical Statistics for H&H

In [ ]:
# =============================================================================
# BASIC SUMMARY STATISTICS
# =============================================================================

print("=== Discharge Summary Statistics ===")
print(f"\nPeriod: {df.index.min().strftime('%Y-%m-%d')} to {df.index.max().strftime('%Y-%m-%d')}")
print(f"Total days: {len(df):,}")

print(f"\n--- Central Tendency ---")
print(f"  Mean:   {df['Q_cfs'].mean():>10,.0f} cfs")
print(f"  Median: {df['Q_cfs'].median():>10,.0f} cfs")

print(f"\n--- Extremes ---")
print(f"  Minimum: {df['Q_cfs'].min():>10,.0f} cfs on {df['Q_cfs'].idxmin().strftime('%Y-%m-%d')}")
print(f"  Maximum: {df['Q_cfs'].max():>10,.0f} cfs on {df['Q_cfs'].idxmax().strftime('%Y-%m-%d')}")

In [ ]:
# =============================================================================
# PERCENTILE STATISTICS
# =============================================================================

percentiles = [10, 25, 50, 75, 90, 95, 99]

print("=== Flow Percentiles ===")
print("\nPercentile  |  Flow (cfs)")
print("-" * 30)

for p in percentiles:
    value = df['Q_cfs'].quantile(p/100)
    print(f"    P{p:<3}    |  {value:>10,.0f}")

---

## Practice Exercises 🎯

Try these exercises to reinforce what you've learned. Attempt them without looking at solutions first, then use your AI assistant if stuck.

### Exercise 1: Parse and Index
1. Load the CSV again into `df2`
2. Rename `time` → `date`, `value` → `Q_cfs`
3. Convert `date` to datetime and set as index
4. Print the date range

In [ ]:
# EXERCISE 1 - YOUR CODE HERE



### Exercise 2: Monthly Plot (Mean and Max)
1. Compute monthly mean AND monthly max discharge
2. Plot both on the same figure with a legend

In [ ]:
# EXERCISE 2 - YOUR CODE HERE



### Exercise 3: Convert to cms and Report Percentile
1. Create a `Q_cms` column
2. Compute the 90th percentile (P90) of `Q_cms`
3. Print it with a clear label

In [ ]:
# EXERCISE 3 - YOUR CODE HERE



### Exercise 4: 14-Day Rolling Statistics
1. Create a 14-day rolling mean from `Q_cfs`
2. Find the date where the rolling mean is maximum
3. Make a plot showing daily flow and 14-day rolling mean

In [ ]:
# EXERCISE 4 - YOUR CODE HERE



### Exercise 5: FDC Function (Challenge)
Write a function `flow_duration_curve(series)` that:
- Takes a pandas Series of flows
- Returns a DataFrame with `exceedance_pct` and `flow`
- Creates an FDC plot

### 🤖 AI Prompt for Exercise 5:
```
Help me write a Python function called 'flow_duration_curve' that:
1. Takes a pandas Series of discharge values
2. Calculates exceedance probability for each sorted value
3. Returns a DataFrame with 'exceedance_pct' and 'flow'
4. Creates a flow duration curve plot
```

In [ ]:
# EXERCISE 5 - YOUR CODE HERE



---

## 🎉 What You Can Now Do

Congratulations! You've learned the core building blocks for time series analysis in H&H:

| Skill | What You Learned |
|-------|------------------|
| **Data Loading** | Read CSV files, inspect data, clean columns |
| **Datetime Handling** | Parse dates, set indexes, time-based slicing |
| **Resampling** | Convert daily → monthly/annual, rolling averages |
| **Visualization** | Hydrographs, box plots, flow duration curves |
| **Statistics** | Percentiles, annual summaries, basic metrics |
| **AI Collaboration** | Use AI to explain, debug, and extend your code |

### What's Next?

Future lessons can build on these skills:
- **Frequency analysis** (annual maxima, fitting distributions)
- **Event detection** (identifying peaks, hydrograph separation)
- **Rainfall-runoff analysis** (comparing precipitation and discharge)

### 🤖 Final AI Prompt: Adapting to Your Data

```
I completed a time series analysis lesson using USGS discharge data.
Now I want to apply the same workflow to [describe your data].

My data is in [format] and has columns for [list columns].

What changes do I need to make to adapt the code?
```